In [1]:
from utils.experimentor import *
from utils.constants import *


In [2]:
config = {
    "dataset_name": Dataset.OGBN_ARXIV,
    "model_type": ModelType.GATV1,
    'num_of_epochs': 10000,
    'num_of_runs': 1,
    'patience_period': 200,
    
    #'should_visualize': False,
    'batch_size': 20000,
    'test_batch_size': 20000,
    'num_workers': 0,
    'force_cpu': False,
    'test_frequency': 10,
    'console_log_freq': 1,
    'do_train_tqdm_logging': False,
    
    'lr': 0.01,
    'num_of_layers': 3,  # PPI has got 42% of nodes with all 0 features - that's why 3 layers are useful
    'num_heads': 1,
    'hidden_size': 256,
    'dropout': 0.25,  
    "use_layer_norm": True,
    "use_saint": False,
    
    #'neighbourhood_degree': 2,
    #'adjacency_mode': AdjacencyMode.OneStep
}

In [3]:
experimentor = Experimentor(config)
experimentor.run()

torch.cuda is available: True
Used Dataset: Dataset.OGBN_ARXIV


ValueError: num_workers option should be non-negative; use num_workers=0 to disable multiprocessing.